In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
import torchvision

In [4]:
from pathlib import Path

In [5]:
testing_folder = Path('../../.fastai/data/mnist_png/testing')
training_folder = Path('../../.fastai/data/mnist_png/training')

[x for x in testing_folder.iterdir() if x.is_dir()]

[Path('../../.fastai/data/mnist_png/testing/5'),
 Path('../../.fastai/data/mnist_png/testing/3'),
 Path('../../.fastai/data/mnist_png/testing/7'),
 Path('../../.fastai/data/mnist_png/testing/4'),
 Path('../../.fastai/data/mnist_png/testing/2'),
 Path('../../.fastai/data/mnist_png/testing/1'),
 Path('../../.fastai/data/mnist_png/testing/9'),
 Path('../../.fastai/data/mnist_png/testing/0'),
 Path('../../.fastai/data/mnist_png/testing/8'),
 Path('../../.fastai/data/mnist_png/testing/6')]

In [6]:
sorted_testing_folder = testing_folder.ls().sorted()
sorted_training_folder = training_folder.ls().sorted()

sorted_training_folder[0], sorted_testing_folder[3]

(Path('../../.fastai/data/mnist_png/training/0'),
 Path('../../.fastai/data/mnist_png/testing/3'))

In [7]:
threes_tensors = [tensor(Image.open(o)) for o in (training_folder/'3').ls().sorted()]
stacked_threes = torch.stack(threes_tensors).float()/255

sevens_tensors = [tensor(Image.open(o)) for o in (training_folder/'7').ls().sorted()]
stacked_sevens = torch.stack(sevens_tensors).float()/255

#reshape the tensors and concatenate them in a new one
test_tensor = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)

stacked_threes.shape, stacked_sevens.shape, test_tensor.shape

(torch.Size([6131, 28, 28]),
 torch.Size([6265, 28, 28]),
 torch.Size([12396, 784]))

In [8]:
def stacked_tensors(path_to_images):
    """A function that returns a stacked Gray Scaled tensor"""
    tensors = [tensor(Image.open(o)) for o in (path_to_images).ls().sorted()]
    stacked_tensors = torch.stack(tensors).float()/255
    return stacked_tensors

In [9]:
# Testing the function works as expected
training_stacked_threes = stacked_tensors(sorted_training_folder[3])
training_stacked_sevens = stacked_tensors(sorted_training_folder[7])

training_stacked_threes.shape, training_stacked_sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [10]:
#Corroborate equality of stacked tensors
if torch.equal(stacked_threes, training_stacked_threes) and torch.equal(stacked_sevens, training_stacked_sevens) :
    print("The tensors are equal.")
else:
    print("The tensors are not equal.")

The tensors are equal.


In [11]:
def squash_tensors(list_of_paths):
    listed_tensors = []
    for i in list_of_paths:
        listed_tensors.append(stacked_tensors(i))
    squashed_tensors = torch.cat(listed_tensors).view(-1, 28*28)
    return squashed_tensors

In [12]:
# Check that the squashed tensors are equal
squashed_threes_and_sevens = squash_tensors([sorted_training_folder[3], sorted_training_folder[7]])

squashed_threes_and_sevens.shape == test_tensor.shape, torch.equal(squashed_threes_and_sevens, test_tensor)

(True, True)

In [13]:
squashed_training_tensors = squash_tensors(sorted_training_folder)

squashed_training_tensors.shape

torch.Size([60000, 784])

In [14]:
valid3s_tensor = torch.stack([tensor(Image.open(o)) for o in (sorted_testing_folder[3]).ls().sorted()]).float()/255
valid7s_tensor = torch.stack([tensor(Image.open(o)) for o in (sorted_testing_folder[7]).ls().sorted()]).float()/255
valid37s_tensor = torch.cat([valid3s_tensor, valid7s_tensor]).view(-1, 28*28)

valid3s_tensor.shape, valid7s_tensor.shape, valid37s_tensor.shape

(torch.Size([1010, 28, 28]),
 torch.Size([1028, 28, 28]),
 torch.Size([2038, 784]))

In [15]:
validation_stacked_threes = stacked_tensors(sorted_testing_folder[3])
validation_stacked_sevens = stacked_tensors(sorted_testing_folder[7])
squashed_valid3and7s_tensor = squash_tensors([sorted_testing_folder[3], sorted_testing_folder[7]])

validation_stacked_threes.shape, validation_stacked_sevens.shape, squashed_valid3and7s_tensor.shape

(torch.Size([1010, 28, 28]),
 torch.Size([1028, 28, 28]),
 torch.Size([2038, 784]))

In [16]:
#Corroborate equality of stacked tensors
if torch.equal(valid3s_tensor, validation_stacked_threes) and torch.equal(valid7s_tensor, validation_stacked_sevens) :
    print("The tensors are equal.")
else:
    print("The tensors are not equal.")

The tensors are equal.


In [17]:
# Check that the squashed tensors are equal
squashed_valid3and7s_tensor.shape == valid37s_tensor.shape, torch.equal(squashed_valid3and7s_tensor, valid37s_tensor)

(True, True)

In [18]:
squashed_validation_tensors = squash_tensors(sorted_testing_folder)

squashed_validation_tensors.shape

torch.Size([10000, 784])

In [7]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.Grayscale(), torchvision.transforms.ToTensor(), torchvision.transforms.Normalize([0.5], [0.5])]
)

In [8]:
training_dataset = torchvision.datasets.ImageFolder((training_folder).as_posix(), transform = transform)
training_dataset.shape()

AttributeError: 'ImageFolder' object has no attribute 'shape'

In [9]:
batchSize = 64
train_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=batchSize, shuffle=True)

train_dataloader

In [10]:
testing_dataset = torchvision.datasets.ImageFolder(testing_folder, transform = transform)
testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=batchSize)

testing_dataloader

In [11]:
dataloaders = {
    "train": train_dataloader,
    "validation": testing_dataloader
}

In [12]:
pytorch_net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 128),
    nn.ReLU(),
    nn.Linear(128, 50),
    nn.ReLU(),
    nn.Linear(50,30),
    nn.ReLU(),
    nn.Linear(30,10),
    nn.LogSoftmax(dim=1))

In [13]:
# nb_epoch is our number of epochs, meaning the number of complete passes through the training dataset.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
lr = 1e-2
nb_epoch = 15

device

device(type='cuda', index=0)

In [14]:
optimizer = torch.optim.SGD(pytorch_net.parameters(), lr=lr)

criterion = nn.NLLLoss()

In [18]:
def train_model(model, criterion, optimizer, dataloaders, num_epochs=10):
    #liveloss = PlotLosses() Live training plot generic API
    model = model.to(device) # Moves and/or casts the parameters and buffers to device.
    
    for epoch in range(num_epochs): # Number of passes through the entire training & validation datasets
        logs = {}
        for phase in ['train', 'validation']: # First train, then validate
            if phase == 'train':
                model.train() # Set the module in training mode
            else:
                model.eval() # Set the module in evaluation mode

            running_loss = 0.0 # keep track of loss
            running_corrects = 0 # count of carrectly classified inputs

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device) # Perform Tensor device conversion
                labels = labels.to(device)

                outputs = model(inputs) # forward pass through network
                loss = criterion(outputs, labels) # Calculate loss

                if phase == 'train':
                    optimizer.zero_grad() # Set all previously calculated gradients to 0
                    loss.backward() # Calculate gradients
                    optimizer.step() # Step on the weights using those gradient w -=  gradient(w) * lr

                _, preds = torch.max(outputs, 1) # Get model's predictions
                running_loss += loss.detach() * inputs.size(0) # multiply mean loss by the number of elements
                running_corrects += torch.sum(preds == labels.data) # add number of correct predictions to total

            epoch_loss = running_loss / len(dataloaders[phase].dataset) # get the "mean" loss for the epoch
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset) # Get proportion of correct predictions
            
            # Logging
            prefix = ''
            if phase == 'validation':
                prefix = 'val_'

            logs[prefix + 'log loss'] = epoch_loss.item()
            logs[prefix + 'accuracy'] = epoch_acc.item()
        print('loss: ', epoch_loss.item(), ' accuracy: ', epoch_acc.item())
        
        #liveloss.update(logs) Update logs
        #liveloss.send()  draw, display stuff

In [19]:
train_model(pytorch_net, criterion, optimizer, dataloaders, nb_epoch)

loss:  0.4052080810070038  accuracy:  0.8804000020027161
loss:  0.34317728877067566  accuracy:  0.8989999890327454
loss:  0.3060593008995056  accuracy:  0.9096999764442444
loss:  0.2595570385456085  accuracy:  0.9241999983787537
loss:  0.23294584453105927  accuracy:  0.9304999709129333
loss:  0.21377748250961304  accuracy:  0.9378999471664429
loss:  0.1968572586774826  accuracy:  0.9429000020027161
loss:  0.17762190103530884  accuracy:  0.9496999979019165
loss:  0.16019994020462036  accuracy:  0.9524999856948853
loss:  0.1489601582288742  accuracy:  0.9549999833106995
loss:  0.1422155797481537  accuracy:  0.9583999514579773
loss:  0.14073260128498077  accuracy:  0.957099974155426
loss:  0.15762324631214142  accuracy:  0.949999988079071
loss:  0.13402201235294342  accuracy:  0.9598000049591064
loss:  0.12043929100036621  accuracy:  0.9657999873161316


In [21]:
# Save the model
#torch.save(pytorch_net, 'models/my_digit_clasifier_3L_97pct.pt')